In [ ]:
class LagsSignificants():
    def __init__(self, dataframe, y_name, n_lags = 5, alpha = 0.05, dropna=True):
        self.significant_lags = []
        self.y_name = y_name
        self.dataframe = dataframe
        self.y = self.dataframe[self.y_name]
        self.n_lags = n_lags
        self.alpha = alpha
        self.dropna = dropna
        self.pac = None
        self.pac_ci = None
    def __set_pac_and_pac_ci(self):
        self.pac, self.pac_ci = sm.tsa.pacf(x=self.y, nlags=self.n_lags, alpha=self.alpha)
    def __significant_lags(self):
        self.__set_pac_and_pac_ci()
        for i in range(1, len(self.pac)):
            if self.pac[i] < self.pac_ci[i][0] - self.pac[i] or self.pac[i] > self.pac_ci[i][1] - self.pac[i]:
                self.significant_lags.append(i)
        return self.significant_lags
    
    def show_significant_lags(self):
        pd.DataFrame(self.pac)[1:].plot(kind='bar', legend='pacf',
                               xlabel='lag', ylabel='pacf');
        print('significantLags:', self.significant_lags)

    def append_lag_variables(self):
        prefix = (self.y_name).replace(')', '')
        df = (self.dataframe).copy()
        for lag in self.significant_lags:
            varNm = '('+prefix+'-'+str(lag)+')'
            df[varNm] = df[self.y_name].shift(lag)
        if self.dropna:
            df.dropna(axis=0, inplace=True)
        return df
    
    def get_ts_significant_lags(self):
        self.__significant_lags()
        df = self.append_lag_variables()
        return df

    

In [ ]:
class TimeSerie(LagsSignificants):
    def __init__(self, ts_name, dataframe):
        self.ts_name = ts_name
        self.dataframe = dataframe
        self.dataframe_with_lags = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
    def get_ts_name(self):
        return self.ts_name
    def get_dataframe(self):
        return self.ts_name
    def get_ts_significant_lags(self, n_lags = 5, alpha = 0.05, dropna=True ):
        super().__init__(self.dataframe, self.ts_name, n_lags, alpha, dropna)
        self.self.dataframe_with_lags =  super().get_ts_significant_lags()
    